In [1]:
#semi supervised learning
from sklearn.base import BaseEstimator
import numpy as np

class SelfLearningModel(BaseEstimator):
    def __init__(self, basemodel, max_iter=200, prob_threshold=0.8):
        self.model = basemodel
        self.max_iter = max_iter
        self.prob_threshold = prob_threshold
        
    def fit(self, X, y):
        labeled_mask = y != -1
        labeled_X, labeled_y = X[labeled_mask], y[labeled_mask]
        unlabeled_X = X[~labeled_mask]
        self.model.fit(labeled_X, labeled_y)
        
        for _ in range(self.max_iter):
            unlabeled_y = self.model.predict(unlabeled_X)
            unlabeled_prob = np.max(self.model.predict_proba(unlabeled_X), axis=1)
            confident_indices = unlabeled_prob > self.prob_threshold
            if not np.any(confident_indices):
                break
            labeled_X = np.vstack([labeled_X, unlabeled_X[confident_indices]])
            labeled_y = np.hstack([labeled_y, unlabeled_y[confident_indices]])
            unlabeled_X = unlabeled_X[~confident_indices]
        return self


In [ ]:
# Random Forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier on the training data
rf_classifier.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = rf_classifier.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


In [6]:
#Decision Tree, Random Forest, and Extremely Randomized Trees
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Initialize the classifiers
dt_classifier = DecisionTreeClassifier(random_state=42)
rf_classifier = RandomForestClassifier(n_estimators=1000, random_state=42)
et_classifier = ExtraTreesClassifier(n_estimators=90, random_state=42)

# Train the classifiers
dt_classifier.fit(X_train, y_train)
rf_classifier.fit(X_train, y_train)
et_classifier.fit(X_train, y_train)

# Make predictions
dt_pred = dt_classifier.predict(X_test)
rf_pred = rf_classifier.predict(X_test)
et_pred = et_classifier.predict(X_test)

# Calculate accuracies
dt_accuracy = accuracy_score(y_test, dt_pred)
rf_accuracy = accuracy_score(y_test, rf_pred)
et_accuracy = accuracy_score(y_test, et_pred)

# Print the accuracies
print("Decision Tree Accuracy:", dt_accuracy)
print("Random Forest Accuracy:", rf_accuracy)
print("Extra Trees Accuracy:", et_accuracy)


Decision Tree Accuracy: 0.9666666666666667
Random Forest Accuracy: 0.9833333333333333
Extra Trees Accuracy: 0.9833333333333333


In [7]:
# adaboost 
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the base estimator (weak learner)
base_estimator = DecisionTreeClassifier(max_depth=1)

# Initialize the AdaBoost classifier with DecisionTreeClassifier as the base estimator
ab_classifier = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=100, random_state=42)

# Train the AdaBoost classifier
ab_classifier.fit(X_train, y_train)

# Make predictions
ab_pred = ab_classifier.predict(X_test)

# Calculate accuracy
ab_accuracy = accuracy_score(y_test, ab_pred)

# Print the accuracy
print("AdaBoost Accuracy:", ab_accuracy)


AdaBoost Accuracy: 1.0


/Users/rohansridhar/miniforge3/envs/google/lib/python3.10/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


In [1]:
from bs4 import BeautifulSoup

text = """\xa0@Flip\xa0how are you not ded"""

# Use BeautifulSoup to handle the special characters
soup = BeautifulSoup(text, "html.parser")
clean_text = soup.get_text()

print(clean_text)


 @Flip how are you not ded


In [2]:
import re

def replace_emails(text):
    email_regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    return re.sub(email_regex, '_EM', text)

text = "Send your feedback to example@email.com or contact us at info@company.com"
cleaned_text = replace_emails(text)
print(cleaned_text)


Send your feedback to _EM or contact us at _EM


In [3]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

# Sample text
text = "This is a sample text for generating unigrams."

# Tokenize the text into words
words = word_tokenize(text)

# Generate unigrams
unigrams = list(ngrams(words, 1))

print(unigrams)


[('This',), ('is',), ('a',), ('sample',), ('text',), ('for',), ('generating',), ('unigrams',), ('.',)]


In [4]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

# Sample text
text = "This is a sample text for generating bigrams."

# Tokenize the text into words
words = word_tokenize(text)

# Generate bigrams
bigrams = list(ngrams(words, 2))

print(bigrams)


[('This', 'is'), ('is', 'a'), ('a', 'sample'), ('sample', 'text'), ('text', 'for'), ('for', 'generating'), ('generating', 'bigrams'), ('bigrams', '.')]


In [5]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download WordNet if you haven't already
nltk.download('wordnet')

# Sample text
text = "running shoes are important for running and runners"

# Tokenize the text into words
words = word_tokenize(text)

# Initialize the WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

# Apply lemmatization to each word
lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

# Join the lemmatized words back into a sentence
lemmatized_text = ' '.join(lemmatized_words)

print(lemmatized_text)


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rohansridhar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


running shoe are important for running and runner


In [6]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# Sample text
text = "running shoes are important for running and runners"

# Tokenize the text into words
words = word_tokenize(text)

# Initialize the Porter Stemmer
porter_stemmer = PorterStemmer()

# Apply stemming to each word
stemmed_words = [porter_stemmer.stem(word) for word in words]

# Join the stemmed words back into a sentence
stemmed_text = ' '.join(stemmed_words)

print(stemmed_text)


run shoe are import for run and runner


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Sample text data (you can replace it with your own dataset)
texts = [
    "This is a good movie",
    "The plot was interesting",
    "I did not like the acting",
    "The movie was boring",
    "The acting was excellent"
]
# Corresponding labels
labels = [1, 1, 0, 0, 1]  # 1 for positive sentiment, 0 for negative sentiment

# Step 1: Vectorize the text data using Bag-of-Words representation
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.3, random_state=42)

# Step 3: Train a classifier (Logistic Regression in this case)
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

# Step 4: Make predictions on the test data
y_pred = classifier.predict(X_test)

# Step 5: Evaluate the performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.0
